空間ダイバーシティ シミュレーション
===

In [1]:
# Import
import numpy as np
import matplotlib.pyplot as plt
import sys

np.set_printoptions(precision=4, suppress=True, floatmode="maxprec_equal")
%matplotlib inline
plt.rcParams['figure.figsize'] = (12,4)

In [2]:
Ndata = 1000
SNRdB = 10
N_R = 4
Nsnapshot = 100

In [3]:
# データ生成器

def rndCode(codeSize, Type):
    """
    codeSize: 生成するランダム系列の大きさ（行数ｘ列数）を指定
    Type: タイプ指定 0: [0,1], 1: [-1, 1]
    """
    
    rndCode = np.random.randn(*codeSize)
    
    if rndCode.ndim > 2:
        sys.exit()
    
    dataSize = rndCode.shape
    if len(dataSize) == 1:
        sys.exit()
    
    if Type == 0:
        rndCode = np.where(rndCode <= 0, 0, 1)
    else:
        rndCode = np.where(rndCode <= 0, -1, 1)
    
    return rndCode

# BPSK変調器
def bpskMod(data):
    """
    data: 列ベクトル
    bpskSymbol： 列ベクトル
    """
    bpskSymbol = data.copy()
    np.place(bpskSymbol, bpskSymbol == 0, -1)
    
    return bpskSymbol

In [4]:
data = rndCode((Ndata, 1), 0)
bpskSymbol = bpskMod(data)
Nsymbol = max(bpskSymbol.shape)

In [74]:
def chCoeff(N, Pa):
    """
    N: チャネル係数の数
    Pa: 平均電力
    """
    
    rndPhase = np.random.rand(N,1) * 2 * np.pi
    rylAmp = rylrnd(Pa, N)
    chCoeff = rylAmp * np.exp(1j * rndPhase)
    
    return chCoeff
    
def rylrnd(Pa, L):
    """
    Pa: 平均電力
    L： 乱数の長さ
    """
    
    Npath = 20
    randPhase = np.random.rand(Npath, L) * 2 * np.pi
    s = np.exp(1j * randPhase) * np.sqrt(Pa / Npath)
    rS = abs(np.array([s.sum(axis=0)]).T)
    
    return rS

def awgn(Pn, rn, cn):
    """
    Pn: 雑音電力
    ｒｎ: 行数
    cn: 列数
    """
    n = (np.random.randn(rn, cn) + 1j * np.random.randn(rn, cn)) * np.sqrt(Pn / 2)
    
    return n
    
def snr(s, r):
    """å
    s: SNR計算対象の信号ベクトル（x　✖️　1）
    r:　参照信号ベクトル（y　✖️　1）
    """
    length1 = max(s.shape)
    length2 = max(r.shape)
    length3 = min([length1, length2])
    coef2_tmp = np.corrcoef(s[0:length3], r[0, 0:length3], rowvar=False)
    coef2 = coef2_tmp[0,1]
    
    snr = abs(coef2) ** 2 / (1 - abs(coef2) ** 2)
    
    return snr

In [76]:
outputSNR_MRC = np.zeros((1, Nsnapshot))
outputSNR = np.zeros((1,Nsnapshot))

for snpCo in range(0,Nsnapshot):
    Pn = 10**(-SNRdB/10)
    chCoeffv = chCoeff(N_R, 1)
    rSig = chCoeffv @ bpskSymbol.T + awgn(Pn, N_R, Nsymbol)
    y_MRC = chCoeffv.conj().T @ rSig;
    outputSNR_MRC[0, snpCo] = snr(bpskSymbol, y_MRC)
    y = np.ones((1,N_R)) @ rSig
    outputSNR[0, snpCo] = snr(bpskSymbol, y)

In [77]:
aveOutputSNR_MRC = 10 * np.log10(np.mean(outputSNR_MRC))
aveOutputSNR = 10 * np.log10(np.mean(outputSNR))

In [78]:
aveOutputSNR_MRC

16.043817708276777

In [79]:
aveOutputSNR

10.417546247731286